Create flower Segmentation

In [7]:
import os
from tqdm import tqdm
import numpy as np
from PIL import Image

# Paths
image_dir = "data/jpg"
mask_dir = "data/segmim"
output_dir = "data/segmented"
os.makedirs(output_dir, exist_ok=True)

# Process all images
for i in tqdm(range(1, 8189 + 1)):  # 1-based indexing
    img_name = f"image_{i:05d}.jpg"
    mask_name = f"segmim_{i:05d}.jpg"

    try:
        img = Image.open(os.path.join(image_dir, img_name)).convert("RGB")
        mask = Image.open(os.path.join(mask_dir, mask_name)).convert("L")

        #convert to numpy
        img_np = np.array(img)
        mask_np = np.array(mask)

        mask_bool = mask_np > 128 #convert grayscale to binary (True=white)
        segmented = img_np.copy()
        segmented[~mask_bool] = 0 #set non flower pixels to black

        seg_img = Image.fromarray(segmented)
        seg_img.save(os.path.join(output_dir, f"segmented_{i:05d}.png"))
    except Exception as e:
        print(f"Error processing {img_name}: {e}")


100%|██████████| 8189/8189 [14:46<00:00,  9.24it/s]


Analyze petals

In [1]:
def create_petal_gallery(img, markers):
    petals = []
    unique_labels = np.unique(markers)
    
    for label in unique_labels:
        if label <= 1:  # Skip background and unknown
            continue

        # Create a mask for this label
        mask = (markers == label).astype(np.uint8) * 255

        # Extract the bounding box
        x, y, w, h = cv2.boundingRect(mask)

        # Mask and crop the petal
        masked_img = cv2.bitwise_and(img, img, mask=mask)
        petal_crop = masked_img[y:y+h, x:x+w]

        # Resize to a fixed height (e.g., 100 px), preserve aspect ratio
        scale = 100 / h
        resized = cv2.resize(petal_crop, (int(w * scale), 100))

        petals.append(resized)

    if not petals:
        return None

    # Stack petals horizontally
    gallery = cv2.hconcat(petals)
    return gallery


In [3]:
import cv2
import numpy as np
from PIL import Image
import os
import json
from tqdm import tqdm

# Directories
segmented_dir = "data/segmented"
output_img_dir = "data/contours"
output_petal_dir = "data/petals"
output_json_dir = "data/petal_data"
os.makedirs(output_img_dir, exist_ok=True)
os.makedirs(output_petal_dir, exist_ok=True)
os.makedirs(output_json_dir, exist_ok=True)

for i in tqdm(range(1, 8189 + 1)):  # Loop through all images
    img_name = f"segmented_{i:05d}.png"
    
    try:
        img_path = os.path.join(segmented_dir, img_name)
        img = cv2.imread(img_path)
        if img is None:
            raise FileNotFoundError(f"Image not found: {img_path}")
        
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        _, binary = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)

        # Preprocessing
        kernel = np.ones((3, 3), np.uint8)
        opening = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel, iterations=2)
        sure_bg = cv2.dilate(opening, kernel, iterations=3)

        # Distance transform + sure foreground
        dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
        _, sure_fg = cv2.threshold(dist_transform, 0.5 * dist_transform.max(), 255, 0)
        sure_fg = np.uint8(sure_fg)
        unknown = cv2.subtract(sure_bg, sure_fg)

        # Markers
        _, markers = cv2.connectedComponents(sure_fg)
        markers = markers + 1
        markers[unknown == 255] = 0

        # Watershed
        output = img.copy()
        markers = cv2.watershed(output, markers)
        gallery = create_petal_gallery(img, markers)
        if gallery is not None:
            gallery_out_path = os.path.join(output_petal_dir, f"petal_gallery_{i:05d}.png")
            cv2.imwrite(gallery_out_path, gallery)

        output[markers == -1] = [0, 0, 255]  # Red boundary

        # Count unique petal regions (label > 1)
        unique_labels = np.unique(markers)
        petal_ids = unique_labels[(unique_labels > 1)]
        petal_count = len(petal_ids)

        # Save image
        output_img = Image.fromarray(cv2.cvtColor(output, cv2.COLOR_BGR2RGB))
        output_img.save(os.path.join(output_img_dir, f"contours_{i:05d}.png"))

        # Save to JSON
        petal_data = [{"petal_count": petal_count}]
        with open(os.path.join(output_json_dir, f"petals_{i:05d}.json"), "w") as f:
            json.dump(petal_data, f, indent=4)

    except Exception as e:
        print(f"Error processing {img_name}: {e}")


100%|██████████| 8189/8189 [10:36<00:00, 12.87it/s]
